In [30]:
import pandas as pd
import requests
import sys

from bs4 import BeautifulSoup

In [31]:
inpatient_1 = pd.read_csv("Data/Train_Inpatientdata-1542865627584.csv")
inpatient_2 = pd.read_csv("Data/Test_Inpatientdata-1542969243754.csv")

inpatient_data = pd.concat([inpatient_1, inpatient_2])
inpatient_data = inpatient_data[[i for i in inpatient_data.columns if 'ClmDiagnosisCode' in i 
                or 'ClmAdmitDiagnosisCode' in i]]

inpatient_data.head()

,ClmAdmitDiagnosisCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10
0,7866,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN
1,6186,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29590,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN
3,431,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN
4,78321,042,3051,34400,5856,42732,486,5119,29620,20300,NaN


In [32]:
# Get all the icd 9 codes
icd_9_codes = pd.melt(inpatient_data, value_vars = 
                      inpatient_data.columns).drop("variable", axis=1).drop_duplicates()

# Drop Null counts
icd_9_codes.dropna(inplace = True)
print("Null Counts:", icd_9_codes.isna().sum())
display(icd_9_codes.head())

Null Counts: value    0
dtype: int64


,value
0,7866
1,6186
2,29590
3,431
4,78321


In [36]:
headers = {'User-Agent':
           'Mozilla/5.0 (iPhone; U; CPU iPhone OS 4_2_1 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8G4 Safari/6533.18.5'}
BASE_URL = 'https://www.icd10data.com/search?s='
ATTRS = {'style': 'font-size:1.5rem;margin:.5rem 0 1rem;padding:0 0 .5rem;'}

def get_soup(url, headers=headers):
    req = requests.get(url, headers=headers)
    return BeautifulSoup(req.content)


def lookup_missing_codes(icd_code):
    x = {}
    x['icd_code'] = icd_code
    if icd_code[0].isdigit():
        BASE_URL = 'https://www.findacode.com/code.php?set=ICD-9&c='
    else:
        BASE_URL = 'https://www.findacode.com/code.php?set=HCPCS&c='
    try:
        soup = get_soup(BASE_URL + icd_code)
        x['matched_code'] = icd_code
        x['matched_code_description'] = soup.find(attrs=ATTRS).text
        return x

    except Exception as e:
        for i in range(len(icd_code)):
            truncated_code = icd_code[:-i]
            try:
                soup = get_soup(BASE_URL + truncated_code)
                x['matched_code'] = truncated_code
                x['matched_code_description'] = soup.find(attrs=ATTRS).text
                return x
            except Exception as e:
                continue
    return x

In [37]:
icd_lookup = pd.DataFrame([lookup_missing_codes(i) for i in icd_9_codes.value.head(5).to_list()])

In [38]:
icd_lookup

,icd_code,matched_code
0,7866,7
1,6186,6
2,29590,2
3,431,4
4,78321,7


In [41]:
soup = get_soup(BASE_URL + '7866')

In [42]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<title></title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/images/icd10data_ico.png" id="lnkShortcutIcon" rel="shortcut icon"/>
<link href="/images/icd10data_ico.png" rel="apple-touch-icon"/>
<link href="/c/c?v=ws1fAWRXHKHF9oi_V9nEP1r6RtuNNqQnSGXGpDmUj6A1" rel="stylesheet">
<!-- Google Tag Manager -->
<script>
        (function (w, d, s, l, i) {
            w[l] = w[l] || []; w[l].push({
                'gtm.start':
                    new Date().getTime(), event: 'gtm.js'
            }); var f = d.getElementsByTagName(s)[0],
                j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : ''; j.async = true; j.src =
                    'https://www.googletagmanager.com/gtm.js?id=' + i + dl; f.parentNode.insertBefore(j, f);
        })(window, document, 'script', 'dataLayer', 'GTM-PSG87ZD');</script>
<!-- End Google Ta